In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk import word_tokenize
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import TruncatedSVD
import sklearn.linear_model as lin_mod


In [2]:
def tfid(data):  
    f = TfidfVectorizer(min_df=2, max_features=None, strip_accents='unicode',  analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)
    f.fit(data)
    data_trans = f.transform(data)
    return data_trans

In [3]:
def log_reg(X,y,X_test):        

    
    rd = lin_mod.LogisticRegression(C=1, tol=0.0001,penalty='l2',dual=True, fit_intercept=True, intercept_scaling=1.0, class_weight=None, random_state=None)

    c = cross_val_score(rd, X, y, cv=5, scoring='roc_auc')
    print("5 Fold CV Score: for Logistic ", np.mean(c) , "+/-" , np.std(c)*2)
    print(c)
    rd.fit(X,y)
    out = rd.predict_proba(X_test)[:,1]
    return np.array(out)

In [ ]:

train=pd.read_csv('train.tsv', sep="\t")
train=pd.DataFrame(train)
train=list(train.iloc[:,2])

test = pd.read_csv('test.tsv', sep="\t")
test=pd.DataFrame(test)
test=list(test.iloc[:,2])


y_train=pd.read_csv('train.tsv', sep="\t")
y_train=pd.DataFrame(y_train)
y_train=np.array(y_train.iloc[:,-1])
y_train= y_train.astype(int)

data = train + test 
data = tfid(data)
strat_data= StratifiedKFold(n_splits=5)
kf=strat_data.get_n_splits(y_train)
svd = TruncatedSVD(n_components=250)
svd.fit(data)
data_svd = svd.transform(data)
X_test = data_svd[len(train):]
X_train = data_svd[:len(train)]


pred = list(log_reg(X_train,y_train,X_test))  
testfile= pd.read_csv('test.tsv', sep="\t", na_values=['?'], index_col=1)
pred_df = pd.DataFrame(pred, index=testfile.index, columns=['label'])
pred_df.to_csv('stumble.csv')
